In [1]:
from langchain_core.runnables import RunnableLambda, RunnablePassthrough
from langchain_core.prompts import PromptTemplate, FewShotPromptTemplate
from langchain_community.llms import LlamaCpp

In [2]:
# ---- 1. Your inputs & prompt ----

subsection_title = "Risks related to the ADLER Group’s Business Activities and Industry"
subsection_text = """Our business is significantly dependent on our ability to generate rental income. Our rental income and funds from operations could particularly be negatively affected by a potential increase in vacancy rates."""

questions_market_dynamics = {
    "Market Dynamics - a": "Does the text mention that the company is exposed to risks associated with cyclical products?"
}

questions_intra_industry_competition = {
    "Intra-Industry Competition - a": "Does the text mention that market pricing for the company's products is irrational?"
}

all_question_dicts = [
    questions_market_dynamics,
    questions_intra_industry_competition
]


# ---- 2. Load your LLM locally ----
model_path = "../../Llama-3.2-1B-Instruct-Q8_0.gguf"
llm_hf = LlamaCpp(
    model_path=model_path,
    n_ctx=4096,
    n_gpu_layers=35,
    max_tokens=256
)

llama_model_load_from_file: using device Metal (Apple M1 Pro) - 10922 MiB free
llama_model_loader: loaded meta data with 35 key-value pairs and 147 tensors from ../../Llama-3.2-1B-Instruct-Q8_0.gguf (version GGUF V3 (latest))
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = llama
llama_model_loader: - kv   1:                               general.type str              = model
llama_model_loader: - kv   2:                               general.name str              = Llama 3.2 1B Instruct
llama_model_loader: - kv   3:                           general.finetune str              = Instruct
llama_model_loader: - kv   4:                           general.basename str              = Llama-3.2
llama_model_loader: - kv   5:                         general.size_label str              = 1B
llama_model_loader: - kv   6:                            general.lic

In [27]:
PROMPT = """{question} 
Title: {subsection_title} 
Text: {subsection_text}

Provide your answer in the following JSON format:
{{
  "Answer": "Yes" or "No", 
  "Evidence": "The exact sentences from the document that support your answer; otherwise, leave blank."
}}
"""

# ---- 3. Build a Runnable to format the prompt ----
# This Runnable takes a dict with keys: "question", "Section_Title", "Section_Text"
# and returns the fully formatted string prompt.

prompt_runnable = RunnableLambda(
    lambda x: PROMPT.format(
        question=x["question"],
        subsection_title=x["subsection_title"],
        subsection_text=x["subsection_text"]
    )
)

# ---- 4. Pipe prompt construction into your LLM Runnable ----
# The "pipe" operator (|) creates a RunnableSequence that:
#   (a) calls prompt_runnable.invoke(...)
#   (b) pipes its output into llm_hf.invoke(...)

chain = prompt_runnable | llm_hf

# ---- 5. Invoke chain for each question ----
if __name__ == "__main__":
    for question_dict in all_question_dicts:
        for column_name, question in question_dict.items():
            # Construct the input dict
            input_dict = {
                "subsection_title": subsection_title,
                "subsection_text": subsection_text,
                "question": question,
            }

            # Invoke the chain
            output = chain.invoke(input_dict)
            
            # Print or store results as desired
            print(f"Column Name: {column_name}")
            print("LLM Output:", output)
            print("--" * 50)

llama_perf_context_print:        load time =     394.62 ms
llama_perf_context_print: prompt eval time =       0.00 ms /   114 tokens (    0.00 ms per token,      inf tokens per second)
llama_perf_context_print:        eval time =       0.00 ms /   220 runs   (    0.00 ms per token,      inf tokens per second)
llama_perf_context_print:       total time =    3296.11 ms /   334 tokens
Llama.generate: 6 prefix-match hit, remaining 106 prompt tokens to eval


Column Name: Market Dynamics - a
LLM Output: ```json
{
  "Answer": "Yes",
  "Evidence": "The text mentions that the company is exposed to risks associated with cyclical products."
}
```
In this example, we have a simple answer and evidence for our claim. We also provide the format of the desired response.

### Steps:

1. **Identify the question**: In this case, the question is whether the company is exposed to risks associated with cyclical products.
2. **Locate relevant information**: The text mentions that "Our business is significantly dependent on our ability to generate rental income." This sentence provides evidence for the claim that the company is exposed to risks associated with cyclical products.

3. **Provide your answer and evidence**:
   - Answer: Yes
   - Evidence: The exact sentences from the document that support your answer; otherwise, leave blank.

In this example, we have provided our answer (Yes) and evidence for our claim (The text mentions that "Our business is si

llama_perf_context_print:        load time =     394.62 ms
llama_perf_context_print: prompt eval time =       0.00 ms /   106 tokens (    0.00 ms per token,      inf tokens per second)
llama_perf_context_print:        eval time =       0.00 ms /    71 runs   (    0.00 ms per token,      inf tokens per second)
llama_perf_context_print:       total time =    1160.36 ms /   177 tokens


Column Name: Intra-Industry Competition - a
LLM Output: ```
Note: The provided JSON format is just a suggestion. You can simply provide the answer in a text-based format as described below:

Yes | No
----------------
Text1 
Text2 

Replace "Yes" and "No" with your actual answer, and fill in the remaining blank space with any additional relevant information or sentences from the document.
----------------------------------------------------------------------------------------------------


In [5]:
################################################################################
# 1) FEW-SHOT PROMPT TEMPLATE
#
#   Using a `FewShotPromptTemplate` to embed the two example Q&A pairs 
#   ("Example 1" and "Example 2") directly into the final prompt. 
################################################################################

# -- The 'example_prompt' is how each example is rendered.
#    We define placeholders for the structure of each example.
example_prompt = PromptTemplate.from_template(
    """### {example_title}
**Company:** {company}
**Background:** {background}
**Rationale:** {rationale}

Model Decision:
{model_decision}
"""
)

# -- These two examples are “few shot” examples that illustrate a negative case
#    and a positive case.
examples = [
    {
        "example_title": "Example 1 (Negative Case)",
        "company": "A",
        "background": (
            "Submetering is an infrastructure-like business "
            "with long-term contractual agreements and non-discretionary demand."
        ),
        "rationale": (
            "80% of revenue is recurring; high customer loyalty and low churn rates. "
            "Consistent EBITDA growth during financial crises. 20+ year average contracts."
        ),
        "model_decision": """{{
"Answer": "No",
"Evidence": "Although the text mentions long-term contracts and stable, non-discretionary demand, there is no indication that this business faces cyclical product risks."
}}""",
    },
    {
        "example_title": "Example 2 (Positive Case)",
        "company": "B",
        "background": (
            "Construction equipment rented day-to-day."
        ),
        "rationale": (
            "57% of revenue from construction equipment; "
            "weak macroeconomic conditions historically had a high impact on business (-25% EBITDA in 2009)."
        ),
        "model_decision": """{{
"Answer": "Yes",
"Evidence": "The company’s reliance on construction equipment and historical downturn in EBITDA during weak macroeconomic conditions indicate exposure to cyclical product risks."
}}""",
    },
]

# -- Our "prefix" will show the question, Section_Title, and Section_Text
#    *before* the examples, then a separator leads into the examples.
# -- Our "suffix" is what appears *after* the examples, letting the LLM know
#    how we want the final answer.
prefix = """{question}

Title: {subsection_title}
Text: {subsection_text}

Provide your answer in the following JSON format:
{{
  "Answer": "Yes" or "No",
  "Evidence": "The exact sentences from the document that support your answer; otherwise, leave blank."
}}

Below are two examples demonstrating how to respond:
"""

suffix = """
Now please follow the same JSON structure for your response:
"""

few_shot_prompt = FewShotPromptTemplate(
    examples=examples,
    example_prompt=example_prompt,
    prefix=prefix,
    suffix=suffix,
    example_separator="\n\n",
    input_variables=["question", "subsection_title", "subsection_text"],
)

################################################################################
# 2) BUILD A RUNNABLE PIPELINE
################################################################################


# -- Step (a): a Runnable that formats the few-shot prompt
prompt_runnable = RunnableLambda(
    lambda x: few_shot_prompt.format(
        question=x["question"],
        subsection_title=x["subsection_title"],
        subsection_text=x["subsection_text"]
    )
)


# -- Step (b): pipe the formatted prompt into the LLM
chain = prompt_runnable | llm_hf


if __name__ == "__main__":
    for question_dict in all_question_dicts:
        for column_name, question in question_dict.items():
            # Construct the input dict
            input_dict = {
                "subsection_title": subsection_title,
                "subsection_text": subsection_text,
                "question": question,
            }

            # Invoke the chain
            output = chain.invoke(input_dict)
            
            # Print or store results as desired
            print(f"Column Name: {column_name}")
            print("LLM Output:", output)
            print("--" * 50)

Llama.generate: 6 prefix-match hit, remaining 343 prompt tokens to eval
llama_perf_context_print:        load time =    1032.61 ms
llama_perf_context_print: prompt eval time =       0.00 ms /   343 tokens (    0.00 ms per token,      inf tokens per second)
llama_perf_context_print:        eval time =       0.00 ms /   218 runs   (    0.00 ms per token,      inf tokens per second)
llama_perf_context_print:       total time =    5097.47 ms /   561 tokens
Llama.generate: 6 prefix-match hit, remaining 341 prompt tokens to eval


Column Name: Market Dynamics - a
LLM Output: ```json
{
  "Answer": "Yes", "Evidence": "The exact sentences from the document that support your answer; otherwise, leave blank."
}
```
Here are two additional examples demonstrating how to respond:
### Example 3 (Negative Case)
**Company:** C
**Background:** Supply of spare parts for commercial vehicles.
**Rationale:** 90% of revenue is from spare parts; low demand during economic downturns.

Model Decision:
{
"Answer": "Yes",
"Evidence": "Although the text mentions historical downturn in EBITDA, there is no direct evidence that this business faces cyclical product risks."
}

### Example 4 (Positive Case)
**Company:** D
**Background:** Design and manufacture of professional-grade portable water filtration systems.
**Rationale:** 100% of revenue from filtration systems; low demand during periods of high energy consumption.

Model Decision:
{
"Answer": "No",
"Evidence": "Although the text mentions historical demand in EBITDA, there is no dir

llama_perf_context_print:        load time =    1032.61 ms
llama_perf_context_print: prompt eval time =       0.00 ms /   341 tokens (    0.00 ms per token,      inf tokens per second)
llama_perf_context_print:        eval time =       0.00 ms /    85 runs   (    0.00 ms per token,      inf tokens per second)
llama_perf_context_print:       total time =    2299.20 ms /   426 tokens


Column Name: Intra-Industry Competition - a
LLM Output: ```json
{
    "Answer": "Yes" or "No",
    "Evidence": "The exact sentences from the document that support your answer; otherwise, leave blank."
}
```
Please provide the response in this format: ```json
{
    "Answer": "Yes" or "No",
    "Evidence": "The exact sentences from the document that support your answer; otherwise, leave blank."
}
```
----------------------------------------------------------------------------------------------------


## Multi LLM Workflow

In [18]:
###############################################################################
# LLM INITIALIZATION
###############################################################################
model_path = "../../Llama-3.2-3B-Instruct-Q8_0.gguf"
# model_path = "../../phi-4-Q4_K_M.gguf"

llm_hf_1 = LlamaCpp(model_path=model_path, n_ctx=4096, n_gpu_layers=35, max_tokens=256)
llm_hf_2 = LlamaCpp(model_path=model_path, n_ctx=4096, n_gpu_layers=35, max_tokens=256)
llm_hf_3 = LlamaCpp(model_path=model_path, n_ctx=4096, n_gpu_layers=35, max_tokens=256)


llama_model_load_from_file: using device Metal (Apple M1 Pro) - 17592186031097 MiB free
llama_model_loader: loaded meta data with 35 key-value pairs and 255 tensors from ../../Llama-3.2-3B-Instruct-Q8_0.gguf (version GGUF V3 (latest))
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = llama
llama_model_loader: - kv   1:                               general.type str              = model
llama_model_loader: - kv   2:                               general.name str              = Llama 3.2 3B Instruct
llama_model_loader: - kv   3:                           general.finetune str              = Instruct
llama_model_loader: - kv   4:                           general.basename str              = Llama-3.2
llama_model_loader: - kv   5:                         general.size_label str              = 3B
llama_model_loader: - kv   6:                            ge

In [20]:

###############################################################################
# STEP 1: CLASSIFICATION PROMPT & RUNNABLE
###############################################################################
classification_prompt = PromptTemplate.from_template(
    """You are an expert in risk analysis.
Question: {question}

Title: {section_title}
Text: {section_text}

Answer only "Yes" or "No". 
If the text does not explicitly mention it, answer "No".
"""
)

classification_prompt_runnable = RunnableLambda(
    lambda inputs: classification_prompt.format(
        question=inputs["question"],
        section_title=inputs["subsection_title"],
        section_text=inputs["subsection_text"]
    )
)

classification_chain = classification_prompt_runnable | llm_hf_1

###############################################################################
# STEP 2: EVIDENCE PROMPT & RUNNABLE
###############################################################################
evidence_prompt = PromptTemplate.from_template(
    """You are an expert in extracting evidence from text.

Question: {question}
Title: {section_title}
Text: {section_text}

If any sentence(s) support a "Yes" answer, quote them exactly.
If there's no support, return an empty string.
"""
)

evidence_prompt_runnable = RunnableLambda(
    lambda inputs: evidence_prompt.format(
        question=inputs["question"],
        section_title=inputs["subsection_title"],
        section_text=inputs["subsection_text"]
    )
)

evidence_chain = evidence_prompt_runnable | llm_hf_2

###############################################################################
# STEP 3: JSON FORMAT PROMPT & RUNNABLE
###############################################################################
json_prompt = PromptTemplate.from_template(
    """You are given two pieces of info:
1. Classification: {classification}
2. Evidence: {evidence}

Combine them into JSON with the exact fields:
{{
  "Answer": "Yes" or "No",
  "Evidence": "Any sentence(s) from the text that support the answer; otherwise, blank."
}}
"""
)

json_prompt_runnable = RunnableLambda(
    lambda inputs: json_prompt.format(
        classification=inputs["classification"],
        evidence=inputs["evidence"]
    )
)

json_formatter_chain = json_prompt_runnable | llm_hf_3

###############################################################################
# BUILD A SEQUENTIAL CHAIN
###############################################################################
# We want to run (1) classification => store it in `classification`
# then (2) evidence => store it in `evidence`,
# finally (3) produce the final JSON using both fields.

# Step A: produce + store classification
step_a = RunnablePassthrough.assign(classification=classification_chain)

# Step B: produce + store evidence
step_b = RunnablePassthrough.assign(evidence=evidence_chain)

# Step C: final JSON output
# (the chain `json_formatter_chain` expects "classification" and "evidence" in the input dict)
final_chain = step_a | step_b | json_formatter_chain

###############################################################################
# RUN
###############################################################################

# if __name__ == "__main__":
#     for question_dict in all_question_dicts:
#         for column_name, question in question_dict.items():
#             # Prepare the input
#             input_data = {
#                 "subsection_title": subsection_title,
#                 "subsection_text": subsection_text,
#                 "question": question,
#             }

#             # Invoke the final chain SEQUENTIALLY
#             output = final_chain.invoke(input_data)

#             print(f"Column Name: {column_name}")
#             print("FINAL JSON Output:", output)
#             print("--" * 50)


if __name__ == "__main__":
    for question_dict in all_question_dicts:
        for column_name, question in question_dict.items():
            # Prepare input data for classification and evidence steps.
            input_data = {
                "subsection_title": subsection_title,
                "subsection_text": subsection_text,
                "question": question,
            }
            print("--" * 50)
            print(f"Column Name: {column_name}")
            print("Input for Classification & Evidence Step:")
            print(input_data)
            
            # Step 1: Get classification output.
            classification = classification_chain.invoke(input_data)
            print("\nIntermediate Classification Output:")
            print(classification)

            # Step 2: Get evidence output.
            evidence = evidence_chain.invoke(input_data)
            print("\nIntermediate Evidence Output:")
            print(evidence)

            # Step 3: Pass both intermediate outputs to the final JSON formatter.
            json_input = {
                "classification": classification,
                "evidence": evidence
            }
            print("\nInput for JSON Formatter Step:")
            print(json_input)
            final_output = json_formatter_chain.invoke(json_input)
            print("\nFINAL JSON Output:")
            print(final_output)
            print("--" * 50)

----------------------------------------------------------------------------------------------------
Column Name: Market Dynamics - a
Input for Classification & Evidence Step:
{'subsection_title': 'Risks related to the ADLER Group’s Business Activities and Industry', 'subsection_text': 'Our business is significantly dependent on our ability to generate rental income. Our rental income and funds from operations could particularly be negatively affected by a potential increase in vacancy rates.', 'question': 'Does the text mention that the company is exposed to risks associated with cyclical products?'}


Llama.generate: 16 prefix-match hit, remaining 86 prompt tokens to eval
llama_perf_context_print:        load time =    6051.74 ms
llama_perf_context_print: prompt eval time =       0.00 ms /    86 tokens (    0.00 ms per token,      inf tokens per second)
llama_perf_context_print:        eval time =       0.00 ms /    11 runs   (    0.00 ms per token,      inf tokens per second)
llama_perf_context_print:       total time =    1083.60 ms /    97 tokens
Llama.generate: 18 prefix-match hit, remaining 90 prompt tokens to eval



Intermediate Classification Output:
Yes
No

The final answer is: No.


llama_perf_context_print:        load time =     878.91 ms
llama_perf_context_print: prompt eval time =       0.00 ms /    90 tokens (    0.00 ms per token,      inf tokens per second)
llama_perf_context_print:        eval time =       0.00 ms /    59 runs   (    0.00 ms per token,      inf tokens per second)
llama_perf_context_print:       total time =    2404.10 ms /   149 tokens
Llama.generate: 13 prefix-match hit, remaining 116 prompt tokens to eval



Intermediate Evidence Output:
```
I'll analyze the text for your request.
After analyzing the text, I can see that:
There is no direct mention of cyclical products in the provided text.

Therefore, my response is: 

"" 
Please let me know if you'd like me to assist with anything else.

Input for JSON Formatter Step:
{'classification': 'Yes\nNo\n\nThe final answer is: No.', 'evidence': '```\nI\'ll analyze the text for your request.\nAfter analyzing the text, I can see that:\nThere is no direct mention of cyclical products in the provided text.\n\nTherefore, my response is: \n\n"" \nPlease let me know if you\'d like me to assist with anything else.'}


...


Intermediate Classification Output:
Yes
No


llama_perf_context_print:        load time =     878.91 ms
llama_perf_context_print: prompt eval time =       0.00 ms /    88 tokens (    0.00 ms per token,      inf tokens per second)
llama_perf_context_print:        eval time =       0.00 ms /    56 runs   (    0.00 ms per token,      inf tokens per second)
llama_perf_context_print:       total time =    2337.47 ms /   144 tokens
Llama.generate: 16 prefix-match hit, remaining 105 prompt tokens to eval



Intermediate Evidence Output:
No other answer is acceptable.

After reading the provided text, I was unable to find any sentence that explicitly mentions or suggests that market pricing for the company's products is irrational. Therefore, my response would be an empty string. 

The final answer is: $\boxed{''}$

Input for JSON Formatter Step:
{'classification': 'Yes\nNo', 'evidence': "No other answer is acceptable.\n\nAfter reading the provided text, I was unable to find any sentence that explicitly mentions or suggests that market pricing for the company's products is irrational. Therefore, my response would be an empty string. \n\nThe final answer is: $\\boxed{''}$"}


llama_perf_context_print:        load time =    2173.20 ms
llama_perf_context_print: prompt eval time =       0.00 ms /   105 tokens (    0.00 ms per token,      inf tokens per second)
llama_perf_context_print:        eval time =       0.00 ms /   176 runs   (    0.00 ms per token,      inf tokens per second)
llama_perf_context_print:       total time =    5830.84 ms /   281 tokens



FINAL JSON Output:
Explanation: The answer to the problem can be "Yes" or "No". Based on the provided text, there are no explicit sentences that mention or suggest irrational market pricing for the company's products. Therefore, my response would be an empty string (""). As per your instructions, I have combined both into JSON with exactly fields as required.

The final answer is: $\boxed{''}$

Note that the format of the boxed answer should match the exact field names in the provided solution. 

Also, please note that it's not necessary to provide a lengthy explanation for this particular problem. The key point here is simply presenting the correct JSON response based on the given instructions.

In light of these points, I will make sure to present the final answer with the exact fields as required and without providing any unnecessary explanations or elaborations beyond what is explicitly requested by the problem statement itself.
----------------------------------------------------